In [1]:
%pip install numpy
%pip install pandas plotly


import pandas as pd
from datetime import datetime

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/h2o/miniconda3/envs/torch_cuda/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Users/h2o/miniconda3/envs/torch_cuda/lib/python3.11/site-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/Users/h2o/miniconda3/envs/torch_cuda/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 701, in start


AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/h2o/miniconda3/envs/torch_cuda/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Users/h2o/miniconda3/envs/torch_cuda/lib/python3.11/site-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/Users/h2o/miniconda3/envs/torch_cuda/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 701, in start


AttributeError: _ARRAY_API not found

In [14]:
import pandas as pd

# Load the CSV data assuming '41.csv' is your filename
data = pd.read_csv('126.csv')

# Step 1: Drop the first row (repeatedly dropping first index), maintaining your instructions
data = data.drop(index=0).reset_index(drop=True)
data = data.drop(index=0).reset_index(drop=True)

rows = []

# Step 2: Iterate over each participant's data to create new rows
for index, row in data.iterrows():
    pid = row['PROLIFIC_PID']
    response_id = row['ResponseId']
    group = row.get('Group')  # Assuming 'Group' is a column in your data for logic
    chart_type = row.get('ChartType')  # Assuming 'ChartType' is a column in your data for logic

    # Create a row for 'analogy' with initialized empty 'sketchscore'
    analogy_row = {
        'PROLIFIC_PID': pid,
        'ResponseId': response_id,
        'Technique': 'analogy',
        'ChartType': chart_type
    }

    # Create a row for 'baseline' with initialized empty 'sketchscore'
    baseline_row = {
        'PROLIFIC_PID': pid,
        'ResponseId': response_id,
        'Technique': 'baseline',
        'ChartType': chart_type
    }

    # Assign 'Order' based on 'Group'
    if group == '1':
        analogy_row['Order'] = 'first'
        baseline_row['Order'] = 'second'
    elif group == '2':
        analogy_row['Order'] = 'second'
        baseline_row['Order'] = 'first'

    # Add the created rows to the list
    rows.append(analogy_row)
    rows.append(baseline_row)

# Convert into DataFrame
new_data = pd.DataFrame(rows, columns=['PROLIFIC_PID', 'ResponseId', 'Order', 'Technique', 'ChartType'])

value_cols = [col for col in data.columns if 'value' in col.lower()]
value_message_cols = [col for col in data.columns if 'valuemessage' in col.lower()]

# Create columns based on extracted column names and append them to new data
for col in value_cols + value_message_cols:
    new_data[col] = None

# Step 4: Populate 'Value' and 'ValueMessage' columns in new_data
for index, row in new_data.iterrows():
    # Find corresponding original data row
    original_row = data[(data['PROLIFIC_PID'] == row['PROLIFIC_PID']) & (data['ResponseId'] == row['ResponseId'])]
    
    if not original_row.empty:
        # There should be only one matching row if IDs are unique
        original_row = original_row.iloc[0]

        for col in value_cols + value_message_cols:
            # Assign values from original data to new data
            new_data.at[index, col] = original_row.get(col, None)

# Display the resulting DataFrame
print(new_data.head())


               PROLIFIC_PID         ResponseId   Order Technique  \
0  60f47600a931759ef3d1a505  R_80P2HAsmCHZkbq8  second   analogy   
1  60f47600a931759ef3d1a505  R_80P2HAsmCHZkbq8   first  baseline   
2  6604674670aaf5b1fd5fa623  R_2xxuuFrPu8yIUzp  second   analogy   
3  6604674670aaf5b1fd5fa623  R_2xxuuFrPu8yIUzp   first  baseline   
4  66d70f8a63fd78797e0776e1  R_8Cw0ccQGdR3Ewmh  second   analogy   

        ChartType                                             Values  \
0         Heatmap                                                NaN   
1         Heatmap                                                NaN   
2     StackedArea  6 data categories: Organic layer, topsoil, elu...   
3     StackedArea  6 data categories: Organic layer, topsoil, elu...   
4  ButterflyChart                                                NaN   

                                        ValueMessage  \
0                                                NaN   
1                                             

In [15]:
# Assuming 'new_data' is your final DataFrame from previous steps
# Use the unique 'chartType' values to separate the DataFrame
chart_types = new_data['ChartType'].unique()

# Dictionary to hold each DataFrame separated by 'chartType'
chart_dataframes = {}

# Iterate over each unique 'chartType' and filter the main DataFrame accordingly
for chart in chart_types:
    # Filter the DataFrame for rows with the current 'chartType'
    chart_df = new_data[new_data['ChartType'] == chart].reset_index(drop=True)
    
    # Store the DataFrame in the dictionary
    chart_dataframes[chart] = chart_df

    # Optionally print or display the DataFrame to verify
    print(f"DataFrame for chartType '{chart}':")
    print(chart_df.head(), "\n")

DataFrame for chartType 'Heatmap':
               PROLIFIC_PID         ResponseId   Order Technique ChartType  \
0  60f47600a931759ef3d1a505  R_80P2HAsmCHZkbq8  second   analogy   Heatmap   
1  60f47600a931759ef3d1a505  R_80P2HAsmCHZkbq8   first  baseline   Heatmap   
2  65cf6d8a589a67afcab54e6f  R_2KZ0NRubjDaUzKi  second   analogy   Heatmap   
3  65cf6d8a589a67afcab54e6f  R_2KZ0NRubjDaUzKi   first  baseline   Heatmap   
4  669805578284a188217bd535  R_8kNvAcFzzYy7N81  second   analogy   Heatmap   

  Values ValueMessage Values10 ValueMessage15 Values42  ...  \
0    NaN          NaN      NaN            NaN      NaN  ...   
1    NaN          NaN      NaN            NaN      NaN  ...   
2    NaN          NaN      NaN            NaN      NaN  ...   
3    NaN          NaN      NaN            NaN      NaN  ...   
4    NaN          NaN      NaN            NaN      NaN  ...   

  ValueV2_Click Count                                    ValueMessage636  \
0                   2  How temperature af

In [16]:
chart_types = new_data['ChartType'].unique()

# Dictionary to hold each DataFrame separated by 'chartType'
chart_dataframes = {}

# Iterate over each unique 'chartType' and filter the main DataFrame accordingly
for chart in chart_types:
    # Filter the DataFrame for rows with the current 'chartType'
    chart_df = new_data[new_data['ChartType'] == chart].reset_index(drop=True)
    
    # Remove columns with all None/NaN values
    chart_df = chart_df.dropna(axis=1, how='all')
    
    # Store the cleaned DataFrame in the dictionary
    chart_dataframes[chart] = chart_df

    # Optionally print or display the cleaned DataFrame to verify
    print(f"DataFrame for chartType '{chart}' after dropping all-None columns:")
    print(chart_df.head(), "\n")

DataFrame for chartType 'Heatmap' after dropping all-None columns:
               PROLIFIC_PID         ResponseId   Order Technique ChartType  \
0  60f47600a931759ef3d1a505  R_80P2HAsmCHZkbq8  second   analogy   Heatmap   
1  60f47600a931759ef3d1a505  R_80P2HAsmCHZkbq8   first  baseline   Heatmap   
2  65cf6d8a589a67afcab54e6f  R_2KZ0NRubjDaUzKi  second   analogy   Heatmap   
3  65cf6d8a589a67afcab54e6f  R_2KZ0NRubjDaUzKi   first  baseline   Heatmap   
4  669805578284a188217bd535  R_8kNvAcFzzYy7N81  second   analogy   Heatmap   

                                           Values589  \
0  The data categories are related to different t...   
1  The data categories are related to different t...   
2  This chart represents a warehouse organized by...   
3  This chart represents a warehouse organized by...   
4  Temperatrures in a warehouse storing different...   

                                     ValueMessage594 ValueV1_First Click632  \
0  To keep certain products in specific shelfs, 

In [18]:
# Assuming `chart_dataframes` is the dictionary with `chartType` as keys and DataFrames as values
# Example:
# chart_dataframes = {
#     "Heatmap": df1,
#     "StackedArea": df2,
#     ...
# }

%pip install xlsxwriter

# Specify the filename for the Excel file
excel_filename = 'charts_data.xlsx'

# Use ExcelWriter to open a file and write multiple DataFrames to it
with pd.ExcelWriter(excel_filename, engine='xlsxwriter') as writer:
    for chart_type, chart_df in chart_dataframes.items():
        # Use the chart type as the sheet name
        chart_df.to_excel(writer, sheet_name=chart_type, index=False)

print(f"Data saved to '{excel_filename}' with each chart type in a separate sheet.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 2.7 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.
Data saved to 'charts_data.xlsx' with each chart type in a separate sheet.
